#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [11]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.6 MB/s eta 0:00:00


,planet,apoapsis,apoapsis_uom
0,:Mercury,0.467,unit:AU
1,:Venus,0.728,unit:AU
2,:Earth,1.017,unit:AU
3,:Earth,149597871,unit:KM
4,:Mars,1.666,unit:AU
5,:Jupiter,5.4588,unit:AU
6,:Saturn,9.0412,unit:AU
7,:Uranus,20.11,unit:AU
8,:Neptune,30.33,unit:AU



## Task 1 (1 pt)

Geben Sie alle Zwergplaneten (Instanzen der Klasse dbo:DwarfPlanet) aus und falls vorhanden deren genaue Entsprechung (skos:exactMatch). Ordnen Sie das Ergebnis aufsteigend nach den URIs der Zwergplaneten.

In [12]:
task1 = sparql_select(g,"""
SELECT ?d ?match
WHERE { ?d a dbo:DwarfPlanet .
OPTIONAL { ?d skos:exactMatch ?match . }  
}
ORDER BY ?d
""")
task1

,d,match
0,:Ceres,dbr:1_Ceres
1,:Eris,None
2,:Haumea,None
3,:Makemake,None
4,:Pluto,dbr:Pluto


## Task 2 (1 pt)

Ermitteln Sie alle Sterne, ihr exactMatch, und ihre Masse (geben Sie den Wert und die Maßeinheit aus). Ordnen Sie das Ergebnis nach der URI der Sterne

In [13]:
task2 = sparql_select(g,"""
SELECT ?star ?match ?massVal ?massUoM
WHERE { ?star a dbo:Star ;  
skos:exactMatch ?match ;
v:mass [rdf:value ?massVal ] ;
v:mass [v:uom ?massUoM ] .
}
ORDER BY ?star
""")
task2

,star,match,massVal,massUoM
0,:AlphaCentauriA,wd:Q2090157,1.1,v:SolarMass
1,:AlphaCentauriB,wd:Q1052548,0.9,v:SolarMass
2,:ProximaCentauri,wd:Q14266,0.1221,v:SolarMass
3,:Sun,dbr:Sun,1.9884e+30,unit:KG


## Task 3 (1 pt)

Die Planeten unseres Sonnensystems und ihre jeweilige Anzahl an Monden. Unterscheiden Sie die im RDF-Graph beschriebenen Monde und die im RDF-Graph erfasste Anzahl von Monden. Sortieren Sie nach den Planeten. 

In [14]:
task3 = sparql_select(g,"""
SELECT ?planet ?assertedNo (COUNT(?moon) as ?noOfDescribedMoons) 
WHERE { 
  ?planet a dbo:Planet ;
          v:nrOfMoons ?assertedNo .
  OPTIONAL { 
    ?moon v:orbits ?planet ;
          a dbo:Satellite .
  }
}
GROUP BY ?planet ?assertedNo
ORDER BY ?planet
""")
task3

,planet,assertedNo,noOfDescribedMoons
0,:Earth,1,1
1,:Jupiter,79,4
2,:Mars,2,2
3,:Mercury,0,0
4,:Neptune,14,0
5,:Saturn,82,2
6,:Uranus,27,0
7,:Venus,0,0


## Task 4 (1 pt)

Ermitteln sie für die Planeten in unserem Sonnensystem die durchschnittliche Anzahl an Monden (die auch im RDF-Graph beschrieben sind) pro Planet. 

In [15]:
task4 = sparql_select(g,"""
SELECT ((?nr/?n) as ?avgNoOfDescribedMoons)
WHERE{
{
SELECT (COUNT(?p) AS ?n)
WHERE {?p a dbo:Planet .}
}
{
SELECT (COUNT(?moon) AS ?nr)
WHERE {
?planet a dbo:Planet .
?moon v:orbits ?planet ;
          a dbo:Satellite .
      }
}

}
""")
task4

,avgNoOfDescribedMoons
0,1.125


## Task 5 (1 pt)

Geben Sie die im RDF-Graph verwendeten Klassen und ihre Anzahl an Instanzen aus. Geben Sie nur Klassen mit mindestens 2 Instanzen aus. Ordnen Sie die Ausgabe nach der URI der Klassen. 

In [16]:
task5 = sparql_select(g,"""
SELECT ?class (COUNT(?s) AS ?noOfInstances) WHERE {
  ?s rdf:type ?class .
  Filter (?class != sdo:x)
}
GROUP BY ?class
HAVING ( COUNT(?s) >= 2 )
ORDER BY ?class
""")
task5

,class,noOfInstances
0,dbo:DwarfPlanet,5
1,dbo:Planet,8
2,dbo:Satellite,9
3,dbo:Star,4


## Task 6 (1 pt)

*TODO: add task description*

In [17]:
task6 = sparql_select(g, """SELECT ?class ?prop (COUNT(DISTINCT ?instance) AS ?noOfInstances)
WHERE {
  { ?instance rdf:type dbo:Planet }
  UNION
  { ?instance rdf:type dbo:DwarfPlanet }
  UNION
  { ?instance rdf:type dbo:Satellite }
  ?instance ?prop ?value .
  ?instance rdf:type ?class .
}
GROUP BY ?class ?prop
ORDER BY ?class ?prop
""")
task6

,class,prop,noOfInstances
0,dbo:DwarfPlanet,v:apoapsis,5
1,dbo:DwarfPlanet,v:nrOfMoons,2
2,dbo:DwarfPlanet,v:orbitalPeriod,5
3,dbo:DwarfPlanet,v:orbits,5
4,dbo:DwarfPlanet,v:radius,5
5,dbo:DwarfPlanet,v:temperature,5
6,dbo:DwarfPlanet,rdf:type,5
7,dbo:DwarfPlanet,skos:exactMatch,2
8,dbo:Planet,v:apoapsis,8
9,dbo:Planet,v:mass,1


## Task 7 (1 pt)

Ermitteln Sie alle Sterne, sowie alle Monde. Zu jedem Mond ermitteln Sie auch den Planet, den dieser Mond umrundet. Ordnen Sie die Ausgabe nach der URI der Sterne und Monde. 

In [18]:
task7 = sparql_select(g,"""
SELECT ?x ?y
WHERE {
  { ?x rdf:type dbo:Star }
  UNION
  { ?x rdf:type dbo:Satellite;
v:orbits ?y .

  }
}
ORDER BY ?x ?y
""")
task7

,x,y
0,:AlphaCentauriA,None
1,:AlphaCentauriB,None
2,:Callisto,:Jupiter
3,:Deimos,:Mars
4,:Enceladus,:Saturn
5,:Europa,:Jupiter
6,:Ganymede,:Jupiter
7,:Io,:Jupiter
8,:Mimas,:Saturn
9,:Moon,:Earth


## Task 8 (1 pt)

*TODO: add task description*

In [20]:
task8 = sparql_construct(g, """CONSTRUCT {
?x rdf:type v:Himmelskoerper ;
 v:wirdUmrundetVon ?orbits.
?y v:wirdUmrundetVon ?l.
}
WHERE {
 {?x a dbo:Planet .}
UNION
 {?x a dbo:Satellite .}
UNION
{?x a dbo:Planet .
 ?orbits a ?egal ;
 v:orbits ?x .}
UNION
{?y a dbo:Star .
?l a ?egal ;
v:orbits ?y .}
}
""")
task8

<Graph identifier=Nc618d0993ff14ea886f66ce4adcf02da (<class 'rdflib.graph.Graph'>)>

## Task 9 (1 pt)

Gibt es einen Stern im RDF-Graph, der massereicher als die Sonne ist.

In [21]:
task9 = sparql_ask(g,"""
ASK {
?x a dbo:Star .
?x v:mass [
 rdf:value ?val; 
] .
?x v:mass [
 v:uom v:SolarMass;
]
FILTER (?val > 0.9942000004971)
}
""")
task9

True

## Task 10 (1 pt)

*TODO: add task description*

In [ ]:
# TODO: include and execute the query, display the result